In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications.resnet import ResNet50

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = ResNet50(include_top=False, weights='imagenet', classifier_activation=None, classes=len(train.class_names),
                      pooling='average',input_shape=inputShape)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    dropoutRate = basicUtil.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/ResnetTrials/resnet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 100:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_resnet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'dropoutRate' : dropoutRate,
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_resnet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_resnet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-08-11 03:14:53.765133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-11 03:14:53.765280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-11 03:14:54.620225: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 38s 497ms/step - loss: 5.9262 - f1_score: 0.3623
devScore


2024-08-11 03:18:03.973248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-08-11 03:18:03.973389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 5s 447ms/step - loss: 5.9275 - f1_score: 0.3485
concluding trial  0
trainScore
75/75 [==============================] - 38s 497ms/step - loss: 1.0576 - f1_score: 0.3251
devScore
10/10 [==============================] - 5s 462ms/step - loss: 1.0576 - f1_score: 0.3128
concluding trial  1
redoing trial 2. Model was 111.40226745605469MB.


...

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
0,15,1,0.092416,0.004534,208.70984,NaN,NaN


trainScore
75/75 [==============================] - 38s 502ms/step - loss: 1.3334 - f1_score: 0.5523
devScore
10/10 [==============================] - 5s 443ms/step - loss: 1.3432 - f1_score: 0.5005
concluding trial  4
trainScore
75/75 [==============================] - 38s 498ms/step - loss: 0.9769 - f1_score: 0.3956
devScore
10/10 [==============================] - 5s 435ms/step - loss: 0.9777 - f1_score: 0.3983
concluding trial  5
trainScore
75/75 [==============================] - 39s 507ms/step - loss: 0.7673 - f1_score: 0.9978
devScore
10/10 [==============================] - 5s 472ms/step - loss: 0.7674 - f1_score: 0.9978
concluding trial  6
trainScore
75/75 [==============================] - 38s 501ms/step - loss: 3.7069 - f1_score: 0.5538
devScore
10/10 [==============================] - 5s 442ms/step - loss: 3.6732 - f1_score: 0.5438
concluding trial  7
trainScore
75/75 [==============================] - 39s 506ms/step - loss: 1.0092 - f1_score: 0.7986
devScore
10/10 [=======

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
6,3,1,0.537795,0.000804,0.027132,0.997800,0.997800
3,10,1,0.352751,0.002683,0.042837,0.997116,0.997117
2,2,1,0.936185,0.001474,0.010130,0.993655,0.993655
8,6,1,0.068898,0.007447,0.239222,0.798626,0.781772
7,3,1,0.788682,0.005227,319.640455,0.553836,0.543787
4,6,1,0.225292,0.000598,155.387385,0.552345,0.500547
5,6,1,0.628325,0.000332,67.925771,0.395587,0.398335
0,11,4,0.663661,0.002905,15.502069,0.362264,0.348507
1,4,1,0.744880,0.000623,53.754732,0.325137,0.312808


Final Model


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,3,1,0.537795,0.000804,0.027132,92.837814,0.9978,0.9978,0.9978
